<a href="https://colab.research.google.com/github/stevejj4/Apache-Spark/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Unsupervisedd learning algorithm
# K-means
# Complexity of the iteration
# single linkage
# complete linkage
# average linkage
# ward linkage
# using spark because of big data


In [1]:
!pip install pyspark
!pip install findspark # simplifies the process of using Apache Spark python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=409475f25d446a8d9a7c808bee0bab311a700fb9ff9f0f7150e5455805e391ce
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

